## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*2


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([43,48])
x = np.repeat(x,26)
y = twentysix*2

## Creating X and Y values for paths

In [3]:
# X values
xvals = [[43,48]]*26

In [4]:
# Y values
# rotor 7
yvals = [[1,14],[2,26],[3,10],[4,8],[5,7],[6,18],[7,3],[8,24],[9,13],
    [10,25],[11,19],[12,23],[13,2],[14,15],[15,21],[16,6],[17,1],
    [18,9],[19,22],[20,12],[21,16],[22,5],[23,11],[24,17],[25,4],
    [26,20]]

# Rotor Links - Rotor 7 [elephant]

In [5]:
# establish links
links = {
    0:[39],     1:[51],      2:[35],     3:[33],     4:[32],     5:[43],      
    6:[28],     7:[49],      8:[38],     9:[50],    10:[44],    11:[48],   
   12:[27],    13:[40],     14:[46],    15:[31],    16:[26],    17:[34],    
   18:[47],    19:[37],     20:[41],    21:[30],    22:[36],    23:[42],    
   24:[29],    25:[45], 
        
   26:[16],    27:[12],     28:[6],     29:[24],    30:[21],    31:[15],
   32:[4],     33:[3],      34:[17],    35:[2],     36:[22],    37:[19],
   38:[8],     39:[0],      40:[13],    41:[20],    42:[23],    43:[5],
   44:[10],    45:[25],     46:[14],    47:[18],    48:[11],    49:[7],  
   50:[9],     51:[1]
}

## Create Graph and Add Features

In [6]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Hover over points')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [7]:
show(p)

# Adding Outer Ring